In [1]:
# Imports

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import the data

train = pd.read_csv('../data/train.csv', encoding='utf-8')

In [3]:
train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [4]:
train.isnull().sum()

tweet_id     0
tweet        1
sentiment    0
dtype: int64

In [5]:
train.dropna(inplace=True)

In [6]:
train.isnull().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [7]:
# Delete ItemID

#train.drop('ItemID', axis=1, inplace=True)

In [8]:
train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [9]:
import re
from string import punctuation

In [10]:
train['tweet'].head()

0    #sxswnui #sxsw #apple defining language of tou...
1    Learning ab Google doodles! All doodles should...
2    one of the most in-your-face ex. of stealing t...
3    This iPhone #SXSW app would b pretty awesome i...
4    Line outside the Apple store in Austin waiting...
Name: tweet, dtype: object

In [11]:
all_sentiments1 = '\n'.join(train['tweet'])

In [12]:
# all_sentiments1

In [13]:
def remove_hash(text):
    hash_pat = r'#'
    return re.sub(hash_pat, ' ', text)
def remove_mention(text):
    mention_pat = r'@mention|@[a-z]+'
    return re.sub(mention_pat, ' ', text, flags=re.I)
def remove_short_link(text):
    short_link_pat = r"bit\.ly/[a-z0-9/\-:\.=%;,\+\*())&\$!@\[\]#\?~_\.']*"
    return re.sub(short_link_pat, ' ', text, flags=re.I) 
def remove_http_link(text):
    link_permit = r"[a-z0-9/\-:\.=%;,\+\*())&\$!@\[\]#\?~_\.']"
    http_link_pat = r"http[s]?://"+link_permit+"+|//"+link_permit+"+|[\w\.]+\.[a-z]+/"+link_permit+"+" 
    return re.sub(http_link_pat, ' ', text, flags=re.I)
def remove_sub_link(text):
    link_pat = r'{link}'
    return re.sub(link_pat, ' ', text, flags=re.I)
def remove_html_char(text):
    html_char_pat = r'&[a-z]+;'
    return re.sub(html_char_pat, ' ', text, flags=re.I)    
def remove_date(text):
    pipe = r'|'
    date_pat_mon = str()
    months =   [r'January',
                r'February',
                r'March',
                r'April',
                r'May',
                r'June',
                r'July',
                r'August',
                r'September',
                r'October',
                r'November',
                r'December']
    for month in months:
        date_pat_mon = date_pat_mon + month+r' \d\d, \d\d\d\d|'
        date_pat_mon = date_pat_mon + month[:3]+r' \d\d, \d\d\d\d|'
    date_pat_mon = date_pat_mon[:-1]
#     date_pat_mon
    date_pat =  r'\d\d/\d\d/\d\d\d\d|\d\d/\d\d/\d\d' + pipe + \
                r'\d\d\.\d\d\.\d\d\d\d|\d\d\.\d\d\.\d\d' + pipe + \
                r'\d\d-\d\d-\d\d\d\d|\d\d-\d\d-\d\d' + pipe + \
                r'{}'.format(date_pat_mon)
    date_pat = r'{}'.format(date_pat)
    return re.sub(date_pat, ' ', text, flags=re.I)
def remove_short_date(text):
    pipe = r'|'
    short_date_pat = r'[\d]?\d/\d\d[\d\d]?' + pipe + r'[\d]?\d\.\d\d[\d\d]?'
    short_date_pat = r'{}'.format(short_date_pat)
    return re.sub(short_date_pat, ' ', text)
def remove_time(text):
    pipe = r'|'
    time_pat =  r'\d\d:\d\d:\d\d[ ]?pm'+pipe+\
                r'\d\d:\d\d:\d\d[ ]?am'+pipe+\
                r'\d\d:\d\d:\d\d'+pipe+\
                r'\d\d:\d\d:\d\d'+pipe+\
                r'[\d]?\d:\d\d[ ]?pm'+pipe+\
                r'[\d]?\d:\d\d[ ]?am'+pipe+\
                r'[\d]?\d:\d\d'+pipe+\
                r'[\d]?\d:\d\d'+pipe+\
                r'[\d]?\d.\d\d[ ]?pm'+pipe+\
                r'[\d]?\d.\d\d[ ]?am'+pipe+\
                r'[\d]?\d.\d\d'+pipe+\
                r'[\d]?\d.\d\d'
    time_pat= r'{}'.format(time_pat)
    return re.sub(time_pat,' ', text, flags=re.I)

def remove_sxsw(text):
    sxsw_pat = r"sxsw[a-z]*"
    return re.sub(sxsw_pat, ' ', text, flags=re.I) 

def remove_austin_texas(text):
    austin_pat = r"austin"
    texas_pat = r"texas|tx"
    temp = text
    temp = re.sub(austin_pat, ' ', temp, flags=re.I) 
    temp = re.sub(texas_pat, ' ', temp, flags=re.I) 
    return temp
def remove_punctuation(text):
    punctuation_pat_s = r'\'s'
    punctuation_pat_t = r'\'t'
    punctuation_pat_d = r'\'d'
    punctuation_pat_ve = r'\'ve'
    punctuation_pat_ll = r'\'ll'
    temp = text
    temp = re.sub(punctuation_pat_s, ' ', temp, flags=re.I)
    temp = re.sub(punctuation_pat_t, 't', temp, flags=re.I)
    temp = re.sub(punctuation_pat_d, ' would', temp, flags=re.I)
    temp = re.sub(punctuation_pat_ve, ' have', temp, flags=re.I)
    temp = re.sub(punctuation_pat_ll, ' will', temp, flags=re.I)
    return temp
def remove_rt(text):
    rt_pat = r'RT'
    return re.sub(rt_pat, ' ', text )

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return re.sub(emoji_pattern,'', text)
def remove_not_alnum(text):
    """
    remove everything that is not alpha numeric
    """
    only_text = r'[^a-z0-9 ]'
    return re.sub(only_text, ' ', text, flags=re.I)  



In [14]:
def clean_text(text):
    #replace links witn {link}
    text = remove_short_link(text)
    text = remove_http_link(text)
    #remove {link}
    text = remove_sub_link(text)
    #hash
    text = remove_hash(text)
    #mention
    text = remove_mention(text)
    #rt
    text = remove_rt(text)
    #html spl chars
    text = remove_html_char(text)
    #time
    text = remove_time(text)
    #svxm
    text=remove_sxsw(text)
    #texas
    text=remove_austin_texas(text)
    #date
    text = remove_date(text)
    text = remove_short_date(text)
    #punctuation
    text = remove_punctuation(text)
    #emotjis
    text = remove_emojis(text)
    #remove spl chars
    text = remove_not_alnum(text)
    #return lower
    return text.lower()

In [15]:
# '''

# def performCleansingForSentimentalAnalysisWithStopWords(completeText):
# #     print('##### Get Tokens in Lower Case #######')
# #     #Get tokens in lower case
# #     lowercasetokens = getUniqueTokens(completeText.lower())
# #     unique_lowercase_tokens = len(set(lowercasetokens))
# #     print('Lowercase Unique Tokens are ' + format(unique_lowercase_tokens))
# #     print('##### Applying  stemming #######')
# #     #Applying the stemming
# #     unique_lowercase_tokens_stemmed= stemData(lowercasetokens)
# #     len_unique_lowercase_tokens_stemmed = len(unique_lowercase_tokens_stemmed)
# #     print('Lowercase Unique Tokens after steming  ' + format(len_unique_lowercase_tokens_stemmed))
# #     print('##### Applying stop words + removing Hashtags + mentions + Links + Short Links +  #######')
#  #   hashtags = find_hashtags(lowercompleteText)
#  #    mentions = find_mentions(lowercompleteText)
#  #   links = find_links(lowercompleteText)
# #    numbers = find_numbers(lowercompleteText)
# #    emojis = find_emojis(lowercompleteText)
# #    punctuations = list(punctuation)
# #     unique_lowercase_tokens_stemmed_without_stopwords = removeCustomStopWords(lowercompleteText,hashtags,mentions,links,numbers,emojis,punctuations)
# #     return unique_lowercase_tokens_stemmed_without_stopwords
#     lowercompleteText = completeText.lower()
#     import re
#     hastag_pat = r'#[0-9a-z_]+'
#     mentions_pat = r'\@[0-9a-z]+'
#     number_pat = r'-?\d+\.?\d+|-?\d+'
#     links_pat= r'http://\S+|https://\S+'
#     punctuations_pat= r'[.!?\\-]'
#     #short_links_pat =
#     emoji_pattern = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags=re.UNICODE)
#     def remove_hashtags(tweet):
#         return re.sub(hastag_pat, '', tweet)
#     def remove_mentions(tweet):
#         return re.sub(mentions_pat, '', tweet)
#     def remove_numbers(tweet):
#         return re.sub(number_pat, '',tweet)
#     def remove_links(tweet):
#         return re.sub(links_pat, '', tweet)
#     def remove_emojis(tweet):
#         return re.sub(emoji_pattern, '', tweet)
#     def remove_shortLinks(tweet):
#         return re.sub(short_links_pat, '', tweet)
#     def remove_punctuations(tweet):
#         return re.sub(punctuations_pat, '', tweet)
#     def find_hashtags(tweet):
#         return re.findall(hastag_pat, tweet)
#     def find_mentions(tweet):
#         return re.findall(mentions_pat, tweet, flags=re.I)
#     def find_links(tweet):
#         return re.findall(links_pat, tweet, flags=re.I)
#     def find_numbers(tweet):
#         return re.findall(number_pat, tweet)
#     def find_emojis(tweet):
#         return re.findall(emoji_pattern, tweet)
    
          
#     lowercompleteText = remove_hashtags(lowercompleteText)
#     lowercompleteText = remove_mentions(lowercompleteText)
#     lowercompleteText = remove_numbers(lowercompleteText)
#     lowercompleteText = remove_links(lowercompleteText)
#     lowercompleteText = remove_emojis(lowercompleteText)
#     lowercompleteText = remove_punctuations(lowercompleteText)
#   #  lowercompleteText = getstemData(lowercompleteText)
#     return  lowercompleteText
#     '''

In [16]:
# train['tweet'] = train['tweet'].apply(lambda x:clean_text(x))
train['tweet_ascii'] = train['tweet'].map(lambda x: x.encode("utf-8").decode("utf-8").encode("ascii","ignore"))



In [57]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

def spacy_ner(text):
#     text = text.replace('\n', ' ')
    doc = nlp(text)
    entities = []
    labels = []
    position_start = []
    position_end = []

    for ent in doc.ents:
        if ent.label_ in ['PERSON','ORG','GPE']:
            entities.append(ent)
            labels.append(ent.label_)
    return entities,labels

def fit_ner(df_og,col):
    """The dataframe should have a column named 'text'"""
    print('Fitting Spacy NER model...')
    df = df_og.copy()
    ner = df[col].apply(spacy_ner)
    ner_org = {}
    ner_per = {}
    ner_gpe = {}

    for x in ner:
        #print(list(x))
        for entity, label in zip(x[0],x[1]):
            #print(type(entity.text))
            if label =='ORG':
                ner_org[entity.text] = ner_org.get(entity.text,0) + 1
            elif label=='PERSON':
                ner_per[entity.text] = ner_per.get(entity.text,0) + 1
            else:
                ner_gpe[entity.text] = ner_gpe.get(entity.text,0) + 1

    return {'ORG':ner_org,'PER':ner_per,'GPE':ner_gpe}

In [58]:
# doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

# for ent in doc.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [59]:
# named_entities = fit_ner(train.loc[0,'tweet'])
# spacy_ner("Apple has launched new iPhone. ")
for i in range(5):
    dummy = remove_hash(str(train.loc[i,'tweet_ascii']))
    print("Sent: ",dummy)
    ents = spacy_ner(dummy)
    print(ents)
    print('---')

Sent:  b' sxswnui  sxsw  apple defining language of touch with different dialects becoming smaller'
([], [])
---
Sent:  b'Learning ab Google doodles! All doodles should be light, funny &amp; innovative, with exceptions for significant occasions.  GoogleDoodle  sxsw'
([funny &amp, GoogleDoodle], ['ORG', 'ORG'])
---
Sent:  b'one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At  SXSW, Apple schools the mkt experts&quot;  {link}'
([RT @mention &quot;At, Apple, mkt], ['ORG', 'ORG', 'ORG'])
---
Sent:  b"This iPhone  SXSW app would b pretty awesome if it didn't crash every 10mins during extended browsing.  Fuckit  Illmakeitwork"
([app], ['ORG'])
---
Sent:  b'Line outside the Apple store in Austin waiting for the new iPad  SXSW  {link}'
([Apple, Austin], ['ORG', 'GPE'])
---


In [61]:
fit_ner(train,'tweet')


Fitting Spacy NER model...


{'ORG': {'funny &amp': 1,
  'RT @mention &quot;At': 2,
  'Apple': 1034,
  'mkt': 1,
  'iPad 2': 41,
  'NPR Videos': 1,
  'NU': 1,
  'RT @mention': 7,
  'UberSocial': 26,
  'iPhone': 432,
  'App Store': 57,
  'UberGuide': 54,
  'the @mention Grill': 2,
  'Calyp Casa': 1,
  'iTunes App Store &amp': 1,
  'Music &gt': 1,
  'Social Network': 19,
  'Possibly': 94,
  'Google': 431,
  'b!tchin': 2,
  'Spazmatics': 4,
  'Circles': 88,
  'FTP': 1,
  'Time Warner Cable': 1,
  'SXSW Post-Mobile Social Party': 1,
  'photography gear &amp': 1,
  'sxsw\x89Û\x9d Dejavu': 1,
  "@mention &quot;I've": 2,
  'Intimate Stranger': 1,
  'Contextual Discovery': 1,
  'LaunchRock': 9,
  'Facebook': 34,
  "Google Labs' Fusion Tables": 1,
  'iPad 1. \r\n\r\n': 1,
  'Anonymous': 2,
  'JAM': 2,
  'Industry Party': 4,
  "@mention &quot;It's": 1,
  'RT @mention RT @mention Tweet': 1,
  'iPhone+Wii controller': 2,
  'RT @mention Horror': 1,
  'Discovr': 4,
  'The Industry Party': 3,
  'GSDM &amp': 8,
  'iPads': 40,
  '

In [63]:
train['tweet_ascii'].astype(str).head()

0    b'#sxswnui #sxsw #apple defining language of t...
1    b'Learning ab Google doodles! All doodles shou...
2    b'one of the most in-your-face ex. of stealing...
3    b"This iPhone #SXSW app would b pretty awesome...
4    b'Line outside the Apple store in Austin waiti...
Name: tweet_ascii, dtype: object

In [32]:
def nameent(text):
    
    entities = []
    labels = []

    sentence = nltk.sent_tokenize(text)
    for sent in sentence:
        
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)),binary=False):
            
            if hasattr(chunk,'label'):
                entities.append(' '.join(c[0] for c in chunk))
                labels.append(chunk.label())
            
    entities_labels = list(set(zip(entities,labels)))
    if entities_labels == []:
        return "No name entities"
    entities_df = pd.DataFrame(entities_labels)
    entities_df.columns = ["Entities","Labels"]
    return entities_df

In [40]:
for i in range(5):
    dummy = remove_hash(str(train.loc[i,'tweet_ascii']))
    print("Sent: ",dummy)
    entities_df = nameent(dummy)
    print(entities_df)
    print('---')
# entities_df = nameent("Apple has launched new iPhone. ")
# entities_df

Sent:  b' sxswnui  sxsw  apple defining language of touch with different dialects becoming smaller'
No name entities
---
Sent:  b'Learning ab Google doodles! All doodles should be light, funny &amp; innovative, with exceptions for significant occasions.  GoogleDoodle  sxsw'
       Entities Labels
0  GoogleDoodle    GPE
---
Sent:  b'one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At  SXSW, Apple schools the mkt experts&quot;  {link}'
  Entities        Labels
0     SXSW  ORGANIZATION
1    Apple           GPE
---
Sent:  b"This iPhone  SXSW app would b pretty awesome if it didn't crash every 10mins during extended browsing.  Fuckit  Illmakeitwork"
        Entities        Labels
0         iPhone  ORGANIZATION
1  Illmakeitwork  ORGANIZATION
2           SXSW  ORGANIZATION
3         Fuckit        PERSON
---
Sent:  b'Line outside the Apple store in Austin waiting for the new iPad  SXSW  {link}'
  Entities        Labels
0     SXSW  ORGANIZATION
1   Austin          

In [22]:
entityname(train['tweet'])

NameError: name 'entityname' is not defined

In [45]:
def entityname(text):
    
    entities = []
    labels = []

    sentence = nltk.sent_tokenize(text)
    for sent in sentence:
        
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)),binary=False):
            
            if hasattr(chunk,'label'):
                
                entities.append(' '.join(c[0] for c in chunk))
                labels.append(chunk.label())
            
                entities_labels = list(set(zip(entities,labels)))
                if entities_labels == []:
                    return "No name entities"
                entities_df = pd.DataFrame(entities_labels)
                entities_df.columns = ["Entities","Labels"]
                
                return entities_df

In [46]:
for i in range(5):
    dummy = remove_hash(str(train.loc[i,'tweet_ascii']))
    print("Sent: ",dummy)
    ents = entityname(dummy)
    print(ents)
    print('---')

Sent:  b' sxswnui  sxsw  apple defining language of touch with different dialects becoming smaller'
None
---
Sent:  b'Learning ab Google doodles! All doodles should be light, funny &amp; innovative, with exceptions for significant occasions.  GoogleDoodle  sxsw'
       Entities Labels
0  GoogleDoodle    GPE
---
Sent:  b'one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At  SXSW, Apple schools the mkt experts&quot;  {link}'
  Entities        Labels
0     SXSW  ORGANIZATION
---
Sent:  b"This iPhone  SXSW app would b pretty awesome if it didn't crash every 10mins during extended browsing.  Fuckit  Illmakeitwork"
  Entities        Labels
0   iPhone  ORGANIZATION
---
Sent:  b'Line outside the Apple store in Austin waiting for the new iPad  SXSW  {link}'
  Entities Labels
0   Austin    GPE
---


In [ ]:
all_sentiments = '\n'.join(train['tweet'])

In [ ]:
all_sentiments

In [ ]:
all_sentiments[:50]

In [ ]:
#nltk.download('stopwords')

In [ ]:
#from nltk.corpus import stopwords
import nltk

from string import punctuation
from nltk.stem.porter import *
#punc = list(punctuation)

# tokenize
train['tokenized_text'] = [nltk.word_tokenize(x) for x in train['tweet']]


In [ ]:
#train['tokenized_text'] = train['tokenized_text'].apply(lambda row: [word for word in row if word not in punc])

#train['tokenized_text'] = train['tokenized_text'].apply(lambda row: [word for word in row if word not in punc])

#train['tweet'] = train['tweet'].apply(lambda row: [word for word in row if word not in punc])

In [ ]:
train['tokenized_text']

In [ ]:
#stemmer = PorterStemmer()

#train['tokenized_text'] = train['tokenized_text'].apply(lambda x: [stemmer.stem(i) for i in x])
train['tokenized_text'] = train['tokenized_text'].apply(lambda x: ' '.join(x))


In [ ]:
train['tokenized_text']

In [ ]:
entityname(train['tokenized_text'])

In [ ]:
tokenized_tweet =train['tokenized_text'].apply(lambda x: x.split()) 

In [ ]:

from textblob import TextBlob

# list to store polarities
tb_polarity = []

# loop over tweets
for sentence in train['tokenized_text']:
    temp = TextBlob(sentence,analyzer=NaiveBayesAnalyzer())
    tb_polarity.append(temp.sentiment[0])

# new column to store polarity    
train['tb_polarity'] = tb_polarity


In [ ]:
train[['sentiment','tb_polarity']]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
X = cv.fit_transform(train['tokenized_text']).toarray()
y = train['sentiment']

In [ ]:
all_words ='\n'.join([str(text) for text in train['tokenized_text']])
all_words
#all_words = '\n'.join(train['tokenized_text'])

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# all the tweets

# generate wordcloud object
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

# plot wordcloud
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
#tok = TreebankWordTokenizer()

In [ ]:
# Find unique tokens

#unique_tokens = len(set(tok.tokenize(all_sentiments)))

#print(f'Unique unprocessed tokens - {unique_tokens}')

In [ ]:
# Unique lower cased tokens

#unique_lower_case = [t.lower() for t in tok.tokenize(all_sentiments)]
#unique_lc_count = len(set(unique_lower_case))

#print(f'Unique lower case unprocessed tokens - {unique_lc_count}')

In [ ]:
#ps = PorterStemmer()

In [ ]:
# Unique lower cased stemmed tokens

#unique_lc_stemmed = [ps.stem(t) for t in unique_lower_case]
#unique_lc_stem_count = len(set(unique_lc_stemmed))

#print(f'Unique lower case stemmed tokens - {unique_lc_stem_count}')

In [ ]:

# Load stop words

#stw = stopwords.words('english')

In [ ]:
# Unique lower cased stemmed w/o stop wordstokens

#unique_lc_stem_wo_stpwrds = [t for t in unique_lc_stemmed if t not in stw]
#unique_lc_stem_wo_stw_count = len(set(unique_lc_stem_wo_stpwrds))

#print(f'Unique lower case stemmed tokens without stop words - {unique_lc_stem_wo_stw_count}')

In [ ]:
# Create a word cloud

#wc = WordCloud(stopwords=stw, background_color='white', max_words=500).generate(all_sentiments.lower())

In [ ]:
#%matplotlib inline
#plt.figure(figsize=(15, 10))
#plt.clf()
#plt.imshow(wc)
#plt.axis('off')
#plt.show()

## Sentiment Analysis

In [ ]:
# Sentiment Analysis imports

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
# Sentiment categories

plt.figure(figsize=(7, 7))
sns.countplot(train['sentiment'])
plt.title('Target Variable Distribution', fontsize=16, color='navy')
plt.show()

In [ ]:
# Create X and Y
#train['tokenized_text'] = [nltk.word_tokenize(x) for x in train['tokenized_text']]

#X = train['tokenized_text']


In [ ]:
X.head()

In [ ]:
#from imblearn.over_sampling import SMOTE


In [ ]:
#sm=SMOTE(sampling_strategy=0.25,random_state=10)

In [ ]:
y.value_counts(normalize=True)*100

In [ ]:
#X_res, y_res = sm.fit_resample(X, y)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Split data in to train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)

In [ ]:
X_train

In [ ]:
#cv = CountVectorizer()

In [ ]:
#X_train_vec = cv.fit_transform(X_train)

In [ ]:
#from imblearn.over_sampling import RandomOverSampler


In [ ]:
# Creating count vectorizer from training data



In [ ]:
# Instantiate all the models

rf = RandomForestClassifier()
lr = LogisticRegression()
#gb = GradientBoostingClassifier()
xgb= XGBClassifier()

In [ ]:
# Convert my test data into vectors for prediction

#X_test_vec = cv.transform(X_test)

In [ ]:
from sklearn import metrics
from xgboost import XGBClassifier


In [ ]:
# Train the models and check CLF report



for model, name in zip([rf, lr, xgb], ['Random Forest', 'Logistic Regression', 'xgboost']):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'CLF report for {name}')
    score = metrics.f1_score(y_test, y_pred,average='weighted')
    print(score)
    print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

classifier=MultinomialNB(alpha=0.1)

In [ ]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred=sub_classifier.predict(X_test)
    score = metrics.f1_score(y_test, y_pred,average='weighted')
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))
    print(classification_report(y_test, y_pred))

In [ ]:
tf = TfidfVectorizer()


In [ ]:
from sklearn import metrics


In [ ]:
X_train_vec_tf = tf.fit_transform(X_train)
X_test_vec_tf = tf.transform(X_test)

In [ ]:
# Train the models and check CLF report for TFIDF vectorizer

for model, name in zip([rf, lr, gb], ['Random Forest', 'Logistic Regression', 'Gradient Boosting']):
    model.fit(X_train_vec_tf, y_train)
    y_pred = model.predict(X_test_vec_tf)
    print(f'CLF report for {name}')
    score = metrics.f1_score(y_test,y_pred,average='weighted')
    print("f1 score:   %0.3f" % score)
    print(classification_report(y_test, y_pred))

## Cleaning with RE

**List of unwanted items**
1. Hashtags (#)
2. Mentions (@)
3. Links (http://, https://)
4. Short link (t.co, bit.ly)
5. Numbers
6. Emojis
7. Emoticons
8. Punctuations